<a href="https://colab.research.google.com/github/thaisrezendeb/ufes_machineLearning/blob/main/Exercicio9.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

**9) Realize a classificação da base de dados Statlog (Image Segmentation) (disponível em http://archive.ics.uci.edu/ml/datasets/Statlog+%28Image+Segmentation%29) usando o esquema de validação hold-out. Para cada execução, selecione aleatoriamente 80% das amostras para treino e o restante para teste. Execute 5 vezes o treinamento e teste e retorne a acurácia, macro recall e macro precisão média para cada algoritmo.
Faça a classificação usando:**

**a) kNN com métrica de distância Euclidiana. Para selecionar o melhor valor de k divida a base de treinamento em duas partes iguais: uma para treinar e a outra para validar e encontrar o melhor valor de k;**

In [ ]:
import numpy as np
import pandas as pd
from sklearn.model_selection import train_test_split
import datetime as dt

#pd.set_option('mode.chained_assignment', None)

segment = pd.read_csv("segment.dat", sep=" ", header=None)

In [ ]:
######################################################
#Classificação por kNN usando distância de Mahalanobis
######################################################
def vizinho(base_train, base_test, kviz=1, conf=False, nclass=1):
  acuracia_knn = 0
  confusao = np.array(np.zeros((nclass, nclass))).astype(int)
  
  if kviz > 1:
    vizinhos = np.array(np.zeros((kviz,2)), dtype=object)

  #Verifica a distância de cada amostra da base de teste com cada uma das amostras da base de treinamento
  for i, row in base_test.iterrows():
    dist_min = 1000000
    
    if kviz > 1:
      for n in range(0, kviz):
        vizinhos[n][1] = dist_min

    classif_new = row[base_test.shape[1]-1]

    for j, row_next in base_train.iterrows():
      dist = 0
      for k in range(0, base_test.shape[1]-1):
        #print(k)
        dist = dist + np.power(row_next[k]-row[k],2)

      dist = np.sqrt(dist)   
      
      if kviz == 1:
        if dist < dist_min:
          dist_min = dist
          classif_new = row_next[base_test.shape[1]-1]
      else: #Mais de um vizinho, matriz de vizinhos
        for z in range(0, vizinhos.shape[0]):
          if dist < vizinhos[z][1]:
            #print("Deslocando vizinho", i, j, z, dist, vizinhos[z][1])
            #Desloca todos os vizinhos logo abaixo
            for y in range(vizinhos.shape[0]-1, z, -1):
              vizinhos[y][0] = vizinhos[y-1][0]
              vizinhos[y][1] = vizinhos[y-1][1]
            #Substitui valor menor
            vizinhos[z][1] = dist
            vizinhos[z][0] = row_next[base_test.shape[1]-1]
            break

    if kviz > 1:
      viz_df = pd.DataFrame(vizinhos, columns=['classif', 'dist'], index=None)
      classif_new = viz_df['classif'].value_counts().index[0]
      #print(classif_new)

    if row[base_test.shape[1]-1] == classif_new:
      acuracia_knn = acuracia_knn + 1

    if conf == True:
      for m in range(0, nclass):
        for n in range(0, nclass):
           if row[base_test.shape[1]-1] == m and classif_new == n:
              confusao[m][n] = confusao[m][n] + 1
        
  #  print("Classif original de", i, ":", row[6], "nova classif", classif_new)
  acuracia_knn = np.divide(acuracia_knn, base_test.shape[0])

  return acuracia_knn, confusao

def idx_k(x):
  return {
      0 : 1,
      1 : 3,
      2 : 5,
      3 : 7
    }.get(x)

In [ ]:
hold = 5
ac_total = 0
num_class = 7
conf_total = np.array(np.zeros((num_class, num_class))).astype(int)

#validação por hold-out
for i in range(0,hold):
  sel_train = np.random.choice(segment.shape[0], round(segment.shape[0]*0.8), replace=False)
  segment_train = segment.iloc[sel_train]
  segment_test = segment.drop(sel_train)
  #segment_train, segment_test = train_test_split(segment, train_size=0.8)

  size = np.divide(segment_train.shape[0],2).astype(int)
  seg_train1 = segment_train[:size]
  seg_train2 = segment_train[size:]

  #definiçao do valor de k
  ac11,a = vizinho(seg_train1, seg_train2, kviz=1)
  ac21,a = vizinho(seg_train1, seg_train2, kviz=3)
  ac31,a = vizinho(seg_train1, seg_train2, kviz=5)
#  ac41,a = vizinho(seg_train1, seg_train2, kviz=7)

  ac12,a = vizinho(seg_train2, seg_train1, kviz=1)
  ac22,a = vizinho(seg_train2, seg_train1, kviz=3)
  ac32,a = vizinho(seg_train2, seg_train1, kviz=5)
#  ac42,a = vizinho(seg_train2, seg_train1, kviz=7)

  ac1 = (ac11 + ac12)/2
  ac2 = (ac21 + ac22)/2
  ac3 = (ac31 + ac32)/2
#  ac4 = (ac41 + ac42)/2

  print("\nAcurácia NN:")
  print("1-NN: {:.2f}%".format(ac1*100), \
        "\n3-NN: {:.2f}%".format(ac2*100),\
        "\n5-NN: {:.2f}%".format(ac3*100))#,\
        #"\n7-NN: {:.2f}%".format(ac4*100))
  
  #Faz a classificação com o k 
#  k = np.array([ac1, ac2, ac3, ac4]).argmax()
  k = np.array([ac1, ac2, ac3]).argmax()
  k = idx_k(k)

  ac, conf = vizinho(segment_train, segment_test, kviz=k, conf=True, nclass=num_class)
  print("Acurácia para k = {} | hold = {}: {:.2f}%".format(k, i+1, ac*100))
  
  #Faz somatório para apresentar médias
  ac_total = ac_total + ac
  conf_total = conf_total + conf

#Acurácia média
ac_total = ac_total/(hold)
print("\nAcurácia média (hold = {}): {:.2f}%".format(hold, ac_total*100))

#Macro precisão média (7 classes)
conf_total = np.divide(conf_total, hold).astype(int)

p = 0
for i in range(0, num_class):
  if conf_total.sum(axis=1)[i] != 0:
    p = p + (conf_total[i][i] / conf_total.sum(axis=1)[i])
        
p = p/num_class
print("\nMacro precisão média: {:.2f}%".format(p*100))

#Macro recall médio
r = 0
for i in range(0, num_class):
  if conf_total.sum(axis=0)[i] != 0:
    r = r + (conf_total[i][i] / conf_total.sum(axis=0)[i])
        
r = r/num_class
print("\nMacro recall médio: {:.2f}%".format(r*100))


Acurácia NN:
1-NN: 94.37% 
3-NN: 92.37% 
5-NN: 91.23%
Acurácia para k = 1 | hold = 1: 95.67%

Acurácia NN:
1-NN: 94.48% 
3-NN: 91.67% 
5-NN: 90.10%
Acurácia para k = 1 | hold = 2: 95.89%

Acurácia NN:
1-NN: 94.26% 
3-NN: 92.32% 
5-NN: 90.48%
Acurácia para k = 1 | hold = 3: 95.24%

Acurácia NN:
1-NN: 94.97% 
3-NN: 92.80% 
5-NN: 90.96%
Acurácia para k = 1 | hold = 4: 96.75%

Acurácia NN:
1-NN: 93.89% 
3-NN: 92.26% 
5-NN: 90.69%
Acurácia para k = 1 | hold = 5: 96.32%

Acurácia média (hold = 5): 95.97%

Macro precisão média: 82.61%

Macro recall médio: 82.71%


**b) Use o Edit-kNN para classificar com métrica de distância Euclidiana. Pode ser usado o mesmo valor de k da letra a).**

In [ ]:
def edit_knn(base_train, base_test, kviz=1):
  edit_base = pd.DataFrame(columns=base_train.columns)

  if kviz > 1:
    vizinhos = np.array(np.zeros((kviz,2)), dtype=object)

  #Verifica a distância de cada amostra da base de teste com cada uma das amostras da base de treinamento
  for i, row in base_test.iterrows():
    dist_min = 1000000
    
    if kviz > 1:
      for n in range(0, kviz):
        vizinhos[n][1] = dist_min

    classif_new = row[base_test.shape[1]-1]

    for j, row_next in base_train.iterrows():
      if j == i:
        continue

      dist = 0
      for n in range(0, base_test.shape[1]-1):
        dist = dist + np.power(row_next[n]-row[n],2)

      dist = np.sqrt(dist)   
      
      if kviz == 1:
        if dist < dist_min:
          dist_min = dist
          classif_new = row_next[base_test.shape[1]-1]
      else: #Mais de um vizinho, matriz de vizinhos
        for z in range(0, vizinhos.shape[0]):
          if dist < vizinhos[z][1]:
            #print("Deslocando vizinho", i, j, z, dist, vizinhos[z][1])
            #Desloca todos os vizinhos logo abaixo
            for y in range(vizinhos.shape[0]-1, z, -1):
              vizinhos[y][0] = vizinhos[y-1][0]
              vizinhos[y][1] = vizinhos[y-1][1]
            #Substitui valor menor
            vizinhos[z][1] = dist
            vizinhos[z][0] = row_next[base_test.shape[1]-1]
            break

    if kviz > 1:
      viz_df = pd.DataFrame(vizinhos, columns=['classif', 'dist'], index=None)
      classif_new = viz_df['classif'].value_counts().index[0]
    
    if row[base_test.shape[1]-1] != classif_new:
      #Amostra desconhecida, mantém na base de dados
      edit_base = edit_base.append(base_test.loc[i,:])
        
  return edit_base

In [ ]:
hold = 5
num_class = 7
k = 1
ac_total = 0
conf_total = np.array(np.zeros((num_class, num_class))).astype(int)

#validação por hold-out
for i in range(0,hold):
  sel_train = np.random.choice(segment.shape[0], round(segment.shape[0]*0.8), replace=False)
  segment_train = segment.iloc[sel_train]
  segment_test = segment.drop(sel_train)
  #segment_train, segment_test = train_test_split(segment, train_size=0.8)

  size = np.divide(segment_train.shape[0],2).astype(int)
  seg_train1 = segment_train[:size]
  seg_train2 = segment_train[size:]
  
  #print(seg_train1.shape)
  seg_train1 = edit_knn(seg_train1, seg_train1, kviz=k)
  #print(seg_train1.shape)
  seg_train2 = edit_knn(seg_train2, seg_train2, kviz=k)

  seg_edit = pd.concat([seg_train1, seg_train2])
  ac, conf = vizinho(seg_edit, segment_test, kviz=k, conf=True, nclass=num_class)
  
  print("\nQuantidade de amostras antes do edit-kNN: {}".format(segment_train.shape[0]))
  print("Quantidade de amostras depois do edit-kNN: {}".format(seg_edit.shape[0]))
  print("Acurácia edit para k = {} | hold = {}: {:.2f}%".format(k, i+1, ac*100))

  #Faz somatório para apresentar médias
  ac_total = ac_total + ac
  conf_total = conf_total + conf

#Acurácia média
ac_total = ac_total/(hold)
print("\nAcurácia edit média (hold = {}): {:.2f}%".format(hold, ac_total*100))

#Macro precisão média (7 classes)
conf_total = conf_total/(hold)

p = 0
for i in range(0, num_class):
  if conf_total.sum(axis=1)[i] != 0:
    p = p + (conf_total[i][i] / conf_total.sum(axis=1)[i])
        
p = p/num_class
print("\nMacro precisão média: {:.2f}%".format(p*100))

#Macro recall médio
r = 0
for i in range(0, num_class):
  if conf_total.sum(axis=0)[i] != 0:
    r = r + (conf_total[i][i] / conf_total.sum(axis=0)[i])
        
r = r/num_class
print("\nMacro recall médio: {:.2f}%".format(r*100))


Quantidade de amostras antes do edit-kNN: 1848
Quantidade de amostras depois do edit-kNN: 115
Acurácia edit para k = 1 | hold = 1: 35.06%

Quantidade de amostras antes do edit-kNN: 1848
Quantidade de amostras depois do edit-kNN: 110
Acurácia edit para k = 1 | hold = 2: 32.90%

Quantidade de amostras antes do edit-kNN: 1848
Quantidade de amostras depois do edit-kNN: 126
Acurácia edit para k = 1 | hold = 3: 35.28%

Quantidade de amostras antes do edit-kNN: 1848
Quantidade de amostras depois do edit-kNN: 101
Acurácia edit para k = 1 | hold = 4: 30.30%

Quantidade de amostras antes do edit-kNN: 1848
Quantidade de amostras depois do edit-kNN: 104
Acurácia edit para k = 1 | hold = 5: 43.29%

Acurácia edit média (hold = 5): 35.37%

Macro precisão média: 33.24%

Macro recall médio: 37.45%


**c) Compare os resultados, tempos de execução e número de protótipos usados por cada algoritmo. Considerando a distribuição das classes, você considera o valor da acurácia média relevante? Por quê?**

Tempo de execução:
- kNN: >4h
- edit-kNN: ~45min

Percebe-se que com o Edit-kNN o tempo de execução foi menor, o que só foi possível com a redução do número de protótipos. Porém, a redução do custo computacional interferiu significativamente no resultado da acurácia, mantendo aproximadamente 5% das amostras originais.

Considerando que as amostras são selecionadas aleatoriamente, o uso da acurácia média pode fazer com que valores extremos não sejam impeditivos para avaliação do modelo. Sendo assim, o uso do hold-out juntamente com os valores médios oferece uma avaliação melhor da eficiência do modelo.